In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.logging.set_verbosity(tf.logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder

from aif360.sklearn.preprocessing import Reweighing, ReweighingMeta, FairAdapt, LearnedFairRepresentations
from aif360.sklearn.inprocessing import AdversarialDebiasing, ExponentiatedGradientReduction, GridSearchReduction
from aif360.sklearn.postprocessing import CalibratedEqualizedOdds, RejectOptionClassifier, PostProcessingMeta, RejectOptionClassifierCV
from aif360.sklearn.datasets import fetch_adult
from aif360.sklearn.metrics import disparate_impact_ratio, average_odds_error, generalized_fpr
from aif360.sklearn.metrics import generalized_fnr, difference

from glob import glob
import json,os

In [3]:
import json,os
def writejson(path,arr,name):
    try:
        os.mkdir(path+'preds/')
    except:
        pass
    f = open(path+'preds/'+name,'w')
    json.dump(arr.tolist(),f)
    f.close()

In [4]:
path = 'data/datasets/compas/'

In [10]:
train_df = pd.read_csv(path+'train.csv',header=None)

train_df.columns = ['juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count',
                'age', 
                'c_charge_degree', 
                'c_charge_desc',
                'age_cat',
                'sex', 'race',  'is_recid']

train_df.index = train_df['sex']

train_df = train_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

test_df = pd.read_csv(path+'test.csv',header=None)

test_df.columns = ['juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count',
                'age', 
                'c_charge_degree', 
                'c_charge_desc',
                'age_cat',
                'sex', 'race',  'is_recid']

test_df.index = test_df['sex']

test_df = test_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)


X_train = train_df.drop(['is_recid'],axis=1)
X_test = test_df.drop(['is_recid'],axis=1)

y_train = pd.Series(train_df['is_recid'])
y_test = pd.Series(test_df['is_recid'])

y_train = pd.Series(y_train.factorize(sort=True)[0], index=y_train.index)
y_test = pd.Series(y_test.factorize(sort=True)[0], index=y_test.index)

X_merged = pd.concat([X_train,X_test])

ohe = make_column_transformer(
    (OneHotEncoder(sparse=False), X_merged.dtypes == 'object'),
    remainder='passthrough', verbose_feature_names_out=False)

X_merged_temp  = pd.DataFrame(ohe.fit_transform(X_merged), columns=ohe.get_feature_names_out(), index=X_merged.index)

X_train  = pd.DataFrame(ohe.transform(X_train), columns=ohe.get_feature_names_out(), index=X_train.index)
X_test = pd.DataFrame(ohe.transform(X_test), columns=ohe.get_feature_names_out(), index=X_test.index)


#### BASELINE
y_pred_baseline = LogisticRegression(solver='liblinear').fit(X_train, y_train).predict(X_test)
writejson(path,y_pred_baseline,'baseline_pred.json')

#### Adv. Reweighting (Preproc.)

rew = ReweighingMeta(estimator=LogisticRegression(solver='liblinear'),
                 reweigher=Reweighing('sex'))

params = {'estimator__C': [1, 10]}

clf = GridSearchCV(rew, params, scoring='accuracy', cv=5)
clf.fit(X_train, y_train)
y_pred_advrew = clf.predict(X_test)
writejson(path,y_pred_advrew,'adv_rew_pred.json')

#### LearnedFairRepresentations (Preproc.)

LFR = LearnedFairRepresentations(prot_attr='sex')
LFR.fit(X_train, y_train)

y_pred_lfr = LFR.predict(X_test)
writejson(path,y_pred_lfr,'lfr_pred.json')

#### Adv Deb. (Inproc.)

adv_deb = AdversarialDebiasing(prot_attr='sex')
adv_deb.fit(X_train, y_train)
adv_deb.score(X_test, y_test)

y_pred_adv_deb = adv_deb.predict(X_test)
writejson(path,y_pred_adv_deb,'adv_deb_pred.json')

adv_deb.sess_.close()

#### ExponentiatedGradientReduction. (Inproc.)

consts = ['DemographicParity','EqualizedOdds','TruePositiveRateParity','FalsePositiveRateParity','ErrorRateParity']
for const in consts:
    EGR = ExponentiatedGradientReduction(prot_attr=['sex_Female','sex_Male'], estimator=LogisticRegression(solver='liblinear'), constraints = const)
    EGR.fit(X_train, y_train)
    print(const,EGR.score(X_test, y_test))
    y_pred_egr = EGR.predict(X_test)
    writejson(path,y_pred_egr,'egr_pred_'+const+'.json')

#### GridSearchReduction. (Inproc.)

consts = ['DemographicParity','EqualizedOdds','TruePositiveRateParity','FalsePositiveRateParity','ErrorRateParity']
for const in consts:
    GSR = GridSearchReduction(prot_attr=['sex_Female','sex_Male'], estimator=LogisticRegression(solver='liblinear'), constraints = const)
    GSR.fit(X_train, y_train)
    print(const,GSR.score(X_test, y_test))
    y_pred_gsr = GSR.predict(X_test)
    writejson(path,y_pred_gsr,'gsr_pred_'+const+'.json')



X_train_temp = X_train.set_index(['sex_Male'], drop = False)
y_train_temp = y_train.copy()
y_train_temp.index = X_train_temp.index

X_test_temp = X_test.set_index(['sex_Male'], drop = False)
y_test_temp = y_test.copy()
y_test_temp.index = X_test_temp.index

#### Cal. Eq.Odds (Postproc.)

consts = ['fnr','fpr','weighted']
for const in consts:
    cal_eq_odds = CalibratedEqualizedOdds(prot_attr='sex_Male', cost_constraint=const)
    postproc = PostProcessingMeta(estimator=LogisticRegression(solver='liblinear'), postprocessor=cal_eq_odds)
    postproc.fit(X_train_temp, y_train_temp)
    print(accuracy_score(y_test_temp, postproc.predict(X_test_temp)))
    y_pred_caleq = postproc.predict(X_test_temp)
    writejson(path,y_pred_caleq,'caleq_pred_'+const+'.json')

#### RejectOptionClassifier (Postproc.)

consts = ['statistical_parity', 'average_odds', 'equal_opportunity']
for const in consts:
    rocv = PostProcessingMeta(LogisticRegression(solver='liblinear'), RejectOptionClassifierCV('sex_Male', scoring=const))
    rocv.fit(X_train_temp, y_train_temp)
    print(accuracy_score(y_test_temp, rocv.predict(X_test_temp)))
    y_pred_rocv = rocv.predict(X_test_temp)
    writejson(path,y_pred_rocv,'rocv_pred_'+const+'.json')


2022-09-12 19:34:01.417812: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-09-12 19:34:01.448042: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2100000000 Hz
2022-09-12 19:34:01.454611: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x559e538c3580 executing computations on platform Host. Devices:
2022-09-12 19:34:01.454773: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>


DemographicParity 0.6706697459584295
EqualizedOdds 0.6780600461893764
TruePositiveRateParity 0.6775981524249423
FalsePositiveRateParity 0.6785219399538106
ErrorRateParity 0.6785219399538106
DemographicParity 0.6678983833718245
EqualizedOdds 0.679445727482679
TruePositiveRateParity 0.6789838337182448
FalsePositiveRateParity 0.6799076212471131
ErrorRateParity 0.679445727482679
0.5284064665127021
0.6655889145496536
0.6785219399538106
0.6817551963048499
0.6762124711316397
0.6526558891454965


#### FairAdapt (Preproc.)

In [46]:
# XY_df = pd.concat([X_train, y_train], axis=1)
# adj_mat = pd.DataFrame(
#     np.zeros((len(train_df.columns), len(train_df.columns)), dtype=int),
#     index=train_df.columns.values,
#     columns=train_df.columns.values
# )

# # Construct the adjacency matrix of the causal graph
# adj_mat.at[
#     ["sex", "age", "native-country"],
#     ["marital-status", "education-num", "workclass", "hours-per-week",
#      "occupation", "annual-income"]
# ] = 1
# adj_mat.at[
#     "marital-status",
#     ["education-num", "workclass", "hours-per-week", "occupation",
#      "annual-income"]
# ] = 1
# adj_mat.at[
#     "education-num",
#     ["workclass", "hours-per-week", "occupation", "annual-income"]
# ] = 1
# adj_mat.at[
#     ["workclass", "hours-per-week", "occupation"],
#     "annual-income"
# ] = 1

# FA = FairAdapt(prot_attr='sex', adj_mat = adj_mat)


# Xf_train, yf_train, Xf_test = FA.fit_transform(X_train, y_train, X_test)
# # y_pred_advrew = clf.predict(X_test)
# # writejson(y_pred_advrew,'adv_rew_pred.json')


#### TAKING TOO LONG TO PROCESS #####

In [139]:
set(X_train.columns).intersection(set(X_test.columns))

{'age',
 'capital-gain',
 'capital-loss',
 'education-num',
 'fnlwgt',
 'hours-per-week'}

In [136]:
X_test.columns

Index(['workclass_?', 'workclass_Federal-gov', 'workclass_Local-gov',
       'workclass_Never-worked', 'workclass_Private', 'workclass_Self-emp-inc',
       'workclass_Self-emp-not-inc', 'workclass_State-gov',
       'workclass_Without-pay', 'education_10th',
       ...
       'native-country_Trinadad&Tobago', 'native-country_United-States',
       'native-country_Vietnam', 'native-country_Yugoslavia', 'age', 'fnlwgt',
       'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'],
      dtype='object', length=107)

In [ ]:
print('still runnung')